# Model Spatial

For Example Testing please run the Section "Model Definition" and "Testing Stride Length" / "Testing Stride Width"

## Dataset Preparation

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import utilities
import configuration
from sklearn.preprocessing import RobustScaler
import math
import scipy
from pickle import dump

# Group the dataframe by 'walk_id'
groups = configuration.df.groupby("walk_id")

# ===========================================================================
# Iterate through groups to create sequences and labels

# Initialize empty lists to store sequences, labels, and IDs
data_sequences = []
data_labels = []
data_ids = []
walk_ids = []
speeds = []

dataset_number = 0

count_gait_cycles = 0
count_gait_cycles_slow = 0
count_gait_cycles_regular = 0
count_gait_cycles_fast = 0

for name, group in groups:

    # Extract the accelerometer data and other relevant columns
    data = group[["acc_x", "acc_y", "acc_z"]].to_numpy()  # Input data
    HeelX = group[["HeelX"]].to_numpy()
    HeelY = group[["HeelY"]].to_numpy()
    events = group[["events"]].to_numpy()

    if group["sensor_side"].iloc[0] == "0_Right" or group["sensor_side"].iloc[0] == "1_Left":

        # Identify Initial Contact (IC) events
        IC = np.where(np.isin(events, [1, 2]))[0]

        # Iterate through each IC event to create sequences and labels
        
        for i in range(0, len(IC) - 2):
            count_gait_cycles += 1
            seq = data[IC[i] : IC[i + 2], :]
            seq = np.array(seq).T  # shape(3,n)

            x_dist = HeelX[IC[i + 2]] - HeelX[IC[i]]
            y_dist = np.abs(HeelY[IC[i + 2]] - HeelY[IC[i]])

            # Ensure distances are positive
            if x_dist < 0:
                x_dist = -1 * x_dist
                y_dist = -1 * y_dist

            stride_length = np.abs(x_dist)

            # Define the coordinates
            A = np.array(
                [HeelX[IC[i]], HeelY[IC[i]]]
            ).squeeze()  # Point  A line-of-progression
            B = np.array(
                [HeelX[IC[i + 2]], HeelY[IC[i + 2]]]
            ).squeeze()  # Point B line-of-progression
            C = np.array(
                [HeelX[IC[i + 1]], HeelY[IC[i + 1]]]
            ).squeeze()  # Point C (D) Hell of different foot

            # Step 1: Vector AB
            AB = B - A

            # Step 2: Unit vector in the direction of AB
            u = AB / np.linalg.norm(AB)

            # Step 3: Vector AC
            AC = C - A

            # Step 4: Projection of AC onto u
            proj_AC_on_u = np.dot(AC, u) * u

            # Step 5: Orthogonal vector from AC to the line AB
            orthogonal_vector = AC - proj_AC_on_u

            # Step 6: Length of the orthogonal vector
            stride_width = np.linalg.norm(orthogonal_vector)

            # Add the sequence to the list  
            data_sequences.append(torch.tensor(seq, dtype=torch.float32))

            # Add the base of support as a label
            # data_labels.append(torch.tensor([stride_width], dtype=torch.float32))

            # Add the stride length as a label
            data_labels.append(torch.tensor([stride_length], dtype=torch.float32))

            data_ids.append(group["id"][group.first_valid_index()])  # len(samples)
            walk_ids.append(group["walk_id"][group.first_valid_index()])
            speeds.extend([group["speed"][group.first_valid_index()]])
            
            if group["speed"][group.first_valid_index()] == "slow":
                count_gait_cycles_slow += 1
            elif group["speed"][group.first_valid_index()] == "regular":
                count_gait_cycles_regular += 1
            elif group["speed"][group.first_valid_index()] == "fast":
                count_gait_cycles_fast += 1

    dataset_number += 1

print(f"Gait cycles: {len(data_sequences)}")#
print(f"Slow gait cycles: {count_gait_cycles_slow}")
print(f"Regular gait cycles: {count_gait_cycles_regular}")
print(f"Fast gait cycles: {count_gait_cycles_fast}")

# ===========================================================================
# Stack data and labels
data_sequences = utilities.pad_and_stack(
    data_sequences
)  # shape(samples,3,sequence_length)
data_labels = torch.vstack(data_labels)  # shape(samples,1)

# ===========================================================================
# Split Datasets for training / unseen testing
train_index, test_index = next(
    configuration.group_split.split(data_sequences, data_labels, data_ids)
)

# ===========================================================================
# Prepare training and test datasets
X = data_sequences[[train_index]]
X_test = data_sequences[[test_index]]

y = data_labels[[train_index]]
y_test = data_labels[[test_index]]

ids = [data_ids[i] for i in train_index]

ids_test_spatial = [data_ids[i] for i in test_index]
walk_ids_test = [walk_ids[i] for i in test_index]
walk_speed_test = [speeds[i] for i in test_index]
# ===========================================================================
# Setup scaler only with training data

# Convert the PyTorch tensor to a NumPy array
x_np = X.numpy()

# Reshape the array to 2D (necessary for the scaler)
n_samples, n_channels, n_features = x_np.shape
x_np_reshaped = x_np.reshape(-1, n_features)

# Apply the RobustScaler
scaler_data = RobustScaler().fit(x_np_reshaped)
x_scaled_np_reshaped = scaler_data.transform(x_np_reshaped)

# Save the scaler
dump(scaler_data, open('checkpoints/scaler_spatial_input_stridelength.pkl', 'wb'))

# Reshape the array back to its original shape
x_scaled_np = x_scaled_np_reshaped.reshape(n_samples, n_channels, n_features)

# Convert the scaled array back to a PyTorch tensor
X = torch.from_numpy(x_scaled_np)

# Scale the target data (labels)
scaler_target = RobustScaler().fit(y)
y = torch.tensor(scaler_target.transform(y))

# Save the scaler
dump(scaler_target, open('checkpoints/scaler_spatial_target_stridelength.pkl', 'wb'))

# ===========================================================================
# Create datasets for testing
# scaling data
x_np = X_test.numpy()
n_samples, n_channels, n_features = x_np.shape
x_np_reshaped = x_np.reshape(-1, n_features)
x_scaled_np_reshaped = scaler_data.transform(x_np_reshaped)
x_scaled_np = x_scaled_np_reshaped.reshape(n_samples, n_channels, n_features)
X_test = torch.from_numpy(x_scaled_np)

# scaling labels
y_test = torch.tensor(scaler_target.transform(y_test))

test_dataset = utilities.CustomDataset(X_test, y_test)

# Get the first three examples from the test dataset
testing_fullwalk_examples = [test_dataset[i] for i in range(3)]

# Dump the testing_fullwalk_examples into a pickle file
with open('example_data/testing_fullwalk_examples_spatial_stridelength.pkl', 'wb') as f:
    dump(testing_fullwalk_examples, f)

test_loader_fullwalk = DataLoader(test_dataset)

## Model Definition

In [ ]:
import torch.nn as nn
import configuration
from torchsummary import summary
from torchview import draw_graph
import graphviz
graphviz.set_jupyter_format('svg')
graphviz.set_default_format('svg')

# Define a custom layer to remove padding from the end of a sequence
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        # Remove the last 'chomp_size' elements from the sequence dimension
        return x[:, :, :-self.chomp_size].contiguous()

# Define a temporal block consisting of two convolutional layers with activation and dropout
class TCNBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TCNBlock, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)  # Remove padding from the end of the sequence
        self.relu1 = nn.ReLU()  # Activation function = rectified linear unit
        self.dropout1 = nn.Dropout(dropout)  # Dropout for regularization
        
        # Second convolutional layer
        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)  # Remove padding from the end of the sequence
        self.relu2 = nn.ReLU()  # Activation function = rectified linear unit
        self.dropout2 = nn.Dropout(dropout)  # Dropout for regularization

        # Sequence of layers in the block
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        
        # Downsample if input and output channels do not match
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()  # Activation function
        self.init_weights()  # Initialize weights

    def init_weights(self):
        # Initialize weights for convolutional layers
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)  # Forward pass through the block
        res = x if self.downsample is None else self.downsample(x)  # Downsample if needed
        return self.relu(out + res)  # Apply residual connection and activation

# Define the Temporal Convolutional Network (TCN) consisting of multiple temporal blocks
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)  # Number of temporal blocks
        for i in range(num_levels):
            dilation_size = 2 ** i  # Exponentially increasing dilation size
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            # Add temporal block to the network
            layers += [TCNBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)  # Sequential container for the network

    def forward(self, x):
        return self.network(x)  # Forward pass through the network

# Define the TCN model for gait spatial analysis
class TCNGaitSpatial(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size=2, dropout=0.2):
        super(TCNGaitSpatial, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout)  # Temporal convolutional network
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Global average pooling
        self.linear = nn.Linear(num_channels[-1], output_size)  # Fully connected layer

    def forward(self, x):
        y1 = self.tcn(x)  # Output shape: (batch_size, num_channels[-1], sequence_length)
        y1_pooled = self.global_avg_pool(y1).squeeze(-1)  # Output shape: (batch_size, num_channels[-1])
        y2 = self.linear(y1_pooled)  # Output shape: (batch_size, output_size)
        return y2

# Model Parameters
input_size = 3  # Number of input channels
output_size = 1  # Number of output channels
num_channels = [16, 32, 64]  # Channel sizes for each layer
kernel_size = 2  # Kernel size for convolutional layers
dropout = 0.2  # Dropout rate

# Instantiate and move the model to the specified device
model = TCNGaitSpatial(input_size, output_size, num_channels, kernel_size, dropout).to(configuration.device)

""" for inputs, labels in test_loader_fullwalk:
        dataset_number += 1
        # Move inputs and labels to the configured device
        inputs, labels = inputs.to(configuration.device), labels.to(
            configuration.device
        )
        summary(model,inputs, depth=10)
        model_graph = draw_graph(model.to(torch.device("cpu")) , inputs.to(torch.device("cpu"),dtype=torch.float32), device='meta', save_graph=True, filename='model_spatial_graph', depth=10)
        break """


## Model Training

In [ ]:
import itertools
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
import tensorflow as tf
import tensorboard
import configuration
import utilities  # Assuming utilities is a module containing required functions


def custom_collate_fn(batch):
    sequences = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    return sequences, labels


def train_model(batch_size, learning_rate, epochs):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()
    val_loss_overall = 0

    log_dir = f"logs/batch_size_{batch_size}_lr_{learning_rate}_epochs_{epochs}"
    writer = tf.summary.create_file_writer(log_dir)

    for epoch in tqdm(range(epochs), desc="Epochs", leave=False):
        model.train()
        split_count = 0
        val_loss_epoch = 0

        for train_index, val_index in configuration.group_kfold.split(X, y, ids):
            split_count += 1

            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            train_dataset = utilities.CustomDataset(X_train, y_train)
            val_dataset = utilities.CustomDataset(X_val, y_val)

            train_loader = DataLoader(
                train_dataset,
                batch_size=batch_size,
                shuffle=True,
                collate_fn=custom_collate_fn,
            )
            val_loader = DataLoader(
                val_dataset, batch_size=batch_size, collate_fn=custom_collate_fn
            )

            total_train_loss_split = 0
            for sequences_batch, labels_batch in train_loader:
                model.train()

                tensor_batch = utilities.pad_and_stack(sequences_batch).to(
                    configuration.device
                )
                labels_batch = torch.squeeze(torch.vstack(labels_batch)).to(
                    configuration.device, dtype=torch.float32
                )

                optimizer.zero_grad()
                output = torch.squeeze(model(tensor_batch))
                loss = loss_fn(output, labels_batch)
                loss.backward()
                optimizer.step()

                total_train_loss_split += loss.item()

            train_loss_split = total_train_loss_split / len(train_loader)

            model.eval()
            total_val_loss_split = 0
            with torch.no_grad():
                for sequences_batch, labels_batch in val_loader:
                    tensor_batch = utilities.pad_and_stack(sequences_batch).to(
                        configuration.device
                    )
                    labels_batch = torch.squeeze(torch.vstack(labels_batch)).to(
                        configuration.device
                    )

                    output = torch.squeeze(model(tensor_batch))
                    loss = loss_fn(output, labels_batch)
                    total_val_loss_split += loss.item()

            avg_val_loss_split = total_val_loss_split / len(val_loader)
            val_loss_epoch += avg_val_loss_split

            with writer.as_default():
                tf.summary.scalar('train_loss_split', train_loss_split, step=epoch)
                tf.summary.scalar('val_loss_split', avg_val_loss_split, step=epoch)
                tf.summary.scalar('loss_diff', train_loss_split - avg_val_loss_split, step=epoch)

            print(
                f"Epoch {epoch + 1}/{epochs} Split {split_count}, Training Loss: {train_loss_split}, Validation Loss: {avg_val_loss_split}, Difference: {train_loss_split - avg_val_loss_split}"
            )

        val_loss_epoch /= split_count
        val_loss_overall += val_loss_epoch

        with writer.as_default():
            tf.summary.scalar('val_loss_epoch', val_loss_epoch, step=epoch)

        print(f"Epoch {epoch + 1}/{epochs}, Mean Validation Loss: {val_loss_epoch}")

    val_loss_overall /= epochs

    os.makedirs("checkpoints", exist_ok=True)
    model_filename = (
        f"checkpoints/model_spatial_bs{batch_size}_lr{learning_rate}_epochs{epochs}.pt"
    )
    torch.save(model.state_dict(), model_filename)
    print(f"Model saved as {model_filename}")

    writer.close()

    return val_loss_overall, model_filename


# Hyperparameter grid
batch_size_options = [4, 8, 12]
learning_rate_options = [1e-3, 1e-4, 1e-5]
epochs_options = [30, 75]

results = []

# Create a summary writer for hyperparameter tuning results
hp_summary_writer = tf.summary.create_file_writer("logs/hyperparameter_tuning")

for batch_size, learning_rate, epochs in itertools.product(batch_size_options, learning_rate_options, epochs_options):
    print(f"Training model with batch size={batch_size}, learning rate={learning_rate}, epochs={epochs}")

    model_performance, model_filename = train_model(batch_size, learning_rate, epochs)
    results.append({
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "epochs": epochs,
        "performance": model_performance,
        "filename": model_filename
    })

    with hp_summary_writer.as_default():
        hp_metric = f"batch_size_{batch_size}_lr_{learning_rate}_epochs_{epochs}"
        tf.summary.scalar(hp_metric, model_performance, step=0)

    print(results)

best_model = min(results, key=lambda x: x["performance"])
print("Best model parameters and performance:", best_model)
print(results)

with hp_summary_writer.as_default():
    tf.summary.text("best_model", str(best_model), step=0)


# Optional: Train with specific parameters if required
# val_loss_overall, model_filename = train_model(1, 1e-03, 15)

## Testing Stride Length

In [ ]:
import pickle
import torch
from torch.utils.data import DataLoader

# Model filename to load
model_filename = "checkpoints/model_spatial_stridelength_bs1_lr1e-05_epochs30_freezing.pt"

# Scaler target
scaler_target = pickle.load(open('checkpoints/scaler_spatial_target_stridelength.pkl', 'rb'))

# Load the model state from the specified checkpoint and set it to evaluation mode
model.load_state_dict(torch.load(model_filename))
model.eval()

# Load Example Data from Pickle (only for testing not for performance evaluation)
with open('example_data/testing_fullwalk_examples_spatial_stridelength.pkl', 'rb') as f:
    test_loader_fullwalk_examples = pickle.load(f)
    test_loader_fullwalk = DataLoader(test_loader_fullwalk_examples)

with torch.no_grad():
    dataset_number = 0
    for inputs, labels in test_loader_fullwalk:
        dataset_number += 1
        # Move inputs and labels to the configured device
        inputs, labels = inputs.to(configuration.device), labels.to(
            configuration.device
        )

        # Get model outputs
        outputs = model(inputs)

        # Detach outputs from the computation graph and move to CPU for plotting
        outputs = outputs.cpu().numpy()
        labels = labels.cpu().numpy()
        
        print(f"Dataset {dataset_number}")
        print(f"Predicted Stride Length: {scaler_target.inverse_transform(outputs[0][0].reshape(-1, 1))}")
        print(f"Actual Stride Length: {scaler_target.inverse_transform(labels[0][0].reshape(-1, 1))}")

## Testing Stride Width

In [ ]:
import pickle
import torch
from torch.utils.data import DataLoader

# Model filename to load
model_filename = "checkpoints/model_spatial_stridewidth_bs1_lr0.001_epochs15_freezing.pt"

# Scaler target
scaler_target = pickle.load(open('checkpoints/scaler_spatial_target_stridewidth.pkl', 'rb'))

# Load the model state from the specified checkpoint and set it to evaluation mode
model.load_state_dict(torch.load(model_filename))
model.eval()

# Load Example Data from Pickle (only for testing not for performance evaluation)
with open('example_data/testing_fullwalk_examples_spatial_stridewidth.pkl', 'rb') as f:
    test_loader_fullwalk_examples = pickle.load(f)
    test_loader_fullwalk = DataLoader(test_loader_fullwalk_examples)

with torch.no_grad():
    dataset_number = 0
    for inputs, labels in test_loader_fullwalk:
        dataset_number += 1
        # Move inputs and labels to the configured device
        inputs, labels = inputs.to(configuration.device), labels.to(
            configuration.device
        )

        # Get model outputs
        outputs = model(inputs)

        # Detach outputs from the computation graph and move to CPU for plotting
        outputs = outputs.cpu().numpy()
        labels = labels.cpu().numpy()
        
        print(f"Dataset {dataset_number}")
        print(f"Predicted Stride Width: {scaler_target.inverse_transform(outputs[0][0].reshape(-1, 1))}")
        print(f"Actual Stride Width: {scaler_target.inverse_transform(labels[0][0].reshape(-1, 1))}")

## Performance Testing

In [ ]:
import custom_statistics
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd

np.seterr(divide='ignore', invalid='ignore')
%config InlineBackend.figure_formats = ['svg']

# optional: define model path manually
model_filename = "checkpoints/model_spatial_stridelength_bs1_lr1e-05_epochs30_freezing.pt"
# model_filename = "checkpoints/model_spatial_stridewidth_bs1_lr0.001_epochs15_freezing.pt"

# Load the model state from the specified checkpoint and set it to evaluation mode
model.load_state_dict(torch.load(model_filename))
model.eval()

# Initialize data lists to store true labels and predicted outputs
data = [[], [], [], [], []]

gt_pd_storage = {"stridelength": {}}

# Ensure no gradients are computed during evaluation
dataset_number = -1
with torch.no_grad():
    for inputs, labels in test_loader_fullwalk:
        dataset_number += 1
        # Move inputs and labels to the configured device
        inputs, labels = inputs.to(configuration.device), labels.to(
            configuration.device
        )

        # Get model outputs
        outputs = model(inputs)

        # Detach outputs from the computation graph and move to CPU for plotting
        outputs = outputs.cpu().numpy()
        labels = labels.cpu().numpy()
        
        act_walk_id = walk_ids_test[dataset_number]

        # fill in for statistics
        if ids_test_spatial[dataset_number] not in gt_pd_storage["stridelength"]:
            gt_pd_storage["stridelength"][ids_test_spatial[dataset_number]] = {
                "gt": {
                    "slow": [],
                    "regular": [],
                    "fast": [],
                    "slow_walks": {},
                    "regular_walks": {},
                    "fast_walks": {},
                },
                "pd": {
                    "slow": [],
                    "regular": [],
                    "fast": [],
                    "slow_walks": {},
                    "regular_walks": {},
                    "fast_walks": {},
                },
            }

        gt_pd_storage["stridelength"][ids_test_spatial[dataset_number]]["gt"][
            walk_speed_test[dataset_number]
        ].append(
            float(
                scaler_target.inverse_transform(
                    np.array(np.squeeze(labels)).reshape(-1, 1)
                )
            )
        )
        gt_pd_storage["stridelength"][ids_test_spatial[dataset_number]]["pd"][
            walk_speed_test[dataset_number]
        ].append(
            float(
                scaler_target.inverse_transform(
                    np.array(np.squeeze(outputs)).reshape(-1, 1)
                )
            )
        )
        
        if act_walk_id not in gt_pd_storage["stridelength"][ids_test_spatial[dataset_number]]["pd"][walk_speed_test[dataset_number]+"_walks"]:
            gt_pd_storage["stridelength"][ids_test_spatial[dataset_number]]["pd"][walk_speed_test[dataset_number]+"_walks"][act_walk_id] = []
            gt_pd_storage["stridelength"][ids_test_spatial[dataset_number]]["gt"][walk_speed_test[dataset_number]+"_walks"][act_walk_id] = []
        
        gt_pd_storage["stridelength"][ids_test_spatial[dataset_number]]["pd"][walk_speed_test[dataset_number]+"_walks"][act_walk_id].append(
            float(
                scaler_target.inverse_transform(
                    np.array(np.squeeze(outputs)).reshape(-1, 1)
                )
            )
        )
        
        gt_pd_storage["stridelength"][ids_test_spatial[dataset_number]]["gt"][walk_speed_test[dataset_number]+"_walks"][act_walk_id].append(
            float(
                scaler_target.inverse_transform(
                    np.array(np.squeeze(labels)).reshape(-1, 1)
                )
            )
        )

statistics = {
    "stridelength": {
        "mean": {
            "gt": {"slow": [], "regular": [], "fast": []},
            "pd": {"slow": [], "regular": [], "fast": []},
        },
        "cv": {
            "gt": {"slow": [], "regular": [], "fast": []},
            "pd": {"slow": [], "regular": [], "fast": []},
        },
        "asymmetry": {
            "gt": {"slow": [], "regular": [], "fast": []},
            "pd": {"slow": [], "regular": [], "fast": []},
        },
        "rmserel": {"slow": {}, "regular": {}, "fast": {}},
    }
}

for parameter in gt_pd_storage:
    for proband in gt_pd_storage[parameter]:
        for sensor in gt_pd_storage[parameter][proband]:
            for speed in ["slow", "regular", "fast"]:
                values = gt_pd_storage[parameter][proband][sensor][speed]
                statistics[parameter]["mean"][sensor][speed].append(np.mean(values))
                statistics[parameter]["cv"][sensor][speed].append(
                    (np.std(values) / np.mean(values)) * 100
                )
                
                # Calculate RMSE rel.
                if sensor == "gt":
                    for values_gt_walk, values_pd_walk in zip(
                        gt_pd_storage[parameter][proband]["gt"][speed+"_walks"].values(),
                        gt_pd_storage[parameter][proband]["pd"][speed+"_walks"].values(),
                    ):
                        values_gt_walk = np.array(values_gt_walk)
                        values_pd_walk = np.array(values_pd_walk)
                        rmse = np.mean(np.sqrt(np.mean((values_gt_walk - values_pd_walk) ** 2)))
                        relative_rmse = (rmse / np.mean(values_gt_walk)) * 100
                        
                        if proband not in statistics[parameter]["rmserel"][speed]:
                            statistics[parameter]["rmserel"][speed][proband] = []
                        
                        statistics[parameter]["rmserel"][speed][proband].append(relative_rmse)


                meanValue0 = np.array(values[1::2])
                meanValue1 = np.array(values[0::2])

                min_length = min(len(meanValue0), len(meanValue1))

                meanValue0 = np.nanmean(meanValue0[:min_length])
                meanValue1 = np.nanmean(meanValue1[:min_length])
                
                if meanValue0 > meanValue1:
                    asymmetrie = 100 * (1 - abs(meanValue1 / meanValue0))
                else:
                    asymmetrie = 100 * (1 - abs(meanValue0 / meanValue1))
                    
                statistics[parameter]["asymmetry"][sensor][speed].append(asymmetrie)

import scipy
import os
import scipy.stats as stats

for parameter in statistics:
    
    rmserel_values = {'slow': [], 'regular': [], 'fast': []}
    
    for speed in statistics[parameter]["rmserel"]:
        rmserel_values_tmp = []
        for proband in statistics[parameter]["rmserel"][speed]:
            rmserel_values_tmp.append(np.mean(statistics[parameter]["rmserel"][speed][proband]))
            
        rmserel_values[speed] = rmserel_values_tmp
        
        print(f"RMSE relative:: Parameter: {parameter}; Speed: {speed}; Probands: {len(rmserel_values_tmp)} ; Mean: {np.mean(rmserel_values_tmp):.4f}; Variance: {np.var(rmserel_values_tmp):.4f}; Std: {np.std(rmserel_values_tmp):.4f}")
    
    for speed, data in zip(rmserel_values, rmserel_values.values()):
        shapiro_test = stats.shapiro(data)
        print(f'Shapiro-Wilk Test {speed}: Statistic={shapiro_test.statistic:.4f}, p-value={shapiro_test.pvalue:.4f}')
    
    # Perform one-way ANOVA
    f_val, p_val = scipy.stats.f_oneway(rmserel_values["slow"], rmserel_values["regular"], rmserel_values["fast"])

    print(f'Parameter: {parameter} ANOVA result: F-value = {f_val:.4f}, P-value = {p_val:.4f}')
    
    plt.figure()
    plt.bar(rmserel_values.keys(), [np.mean(values) for values in rmserel_values.values()], yerr=[np.std(values) for values in rmserel_values.values()])
    plt.xlabel('Speed')
    plt.ylabel('RMSE Relative')
    plt.ylim(0, 60)
    plt.title('RMSE Relative for Different Speeds')
    # Save the plot as svg in the rmse_figures folder
    plt.savefig(f'rmse_figures/rmse_plot_{parameter}.svg', format="svg")
    
    plt.show()
print("here")

def standard_output(metric, parameter):

    overall_gt = np.array(
        statistics[metric][parameter]["gt"]["fast"]
        + statistics[metric][parameter]["gt"]["slow"]
        + statistics[metric][parameter]["gt"]["regular"]
    )
    overall_pd = np.array(
        statistics[metric][parameter]["pd"]["fast"]
        + statistics[metric][parameter]["pd"]["slow"]
        + statistics[metric][parameter]["pd"]["regular"]
    )

    plt.figure()
    plt.scatter(overall_gt, overall_pd)
    #plt.show()

    print("============================================")
    print("Correlation & Statistics")
    print("============================================")
    print("SLOW ============================================")
    print(
        custom_statistics.uniform_statistics(
            np.array(statistics[metric][parameter]["gt"]["slow"]),
            np.array(statistics[metric][parameter]["pd"]["slow"]),
        )
    )
    print("REGULAR ============================================")
    print(
        custom_statistics.uniform_statistics(
            np.array(statistics[metric][parameter]["gt"]["regular"]),
            np.array(statistics[metric][parameter]["pd"]["regular"]),
        )
    )
    print("FAST ============================================")
    print(
        custom_statistics.uniform_statistics(
            np.array(statistics[metric][parameter]["gt"]["fast"]),
            np.array(statistics[metric][parameter]["pd"]["fast"]),
        )
    )
    print("OVERALL ============================================")
    print(custom_statistics.uniform_statistics(overall_gt, overall_pd))


standard_output("stridelength", "mean")
standard_output("stridelength", "cv")
print("asymmetrie ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
standard_output("stridelength", "asymmetry")